In [1]:
import os, sys, json, torch, random
import numpy as np
from pathlib import Path
from PIL import Image

# 固定随机种子
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

sys.path.append("/home/czj/llava15_test/LLaVA")

from llava.model.builder import load_pretrained_model
from llava.data.gqa_loader import GQALoader
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN




/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/torch/cuda/__init__.py:51: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-10-22 16:11:00,773] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because

In [2]:
from gqa_eval.config import get_args
from gqa_eval.seed_utils import set_seed
from gqa_eval.model_loader import load_llava_model
from llava.data.gqa_loader import GQALoader

In [3]:
import argparse
def get_fixed_args():
    # 创建一个 Namespace 对象，手动设置默认值（或你想要的值）
    args = argparse.Namespace(
        # 模型加载
        model_path="/home/czj/llava15_test/llava-v1.5-7b",
        device="cuda",
        dtype="float16",
        load_in_4bit=False,
        load_in_8bit=False,

        # 超参数
        batch_size=2,
        num_workers=8,
        max_new_tokens=64,

        # GPU并行
        multi_gpu=False,

        # mask选项
        mask_visual_token=False,
        mask_ratio=0.1,
        mask_strategy="random",
        mask_token_value=0.0,

        # 数据集
        dataset_path="/home/Dataset/Dataset/GQA",
        split="testdev",
        max_samples=None,
        conv_mode="llava_v1",

        # 输出
        output_dir="./eval_GQA_results/testdev_VQA_100samples",
        save_json=True,
        save_pred_gt=True,

        seed=42
    )
    return args
args = get_fixed_args()

In [4]:
gqa_loader = GQALoader(args.dataset_path, num_workers=args.num_workers, batch_size=args.batch_size)


使用本地 GQA 数据集路径: /home/Dataset/Dataset/GQA


In [5]:
tokenizer, model, image_processor = load_llava_model(args)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


加载模型: /home/czj/llava15_test/llava-v1.5-7b
设备: cuda | dtype=float16 | multi_gpu=False
mask_visual_token=False, strategy=random
模型类型: LLaVA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.54s/it]


In [6]:
import numpy as np
from tqdm import tqdm
from collections import Counter
from gqa_eval.pred_gt_match import compute_match_batch
from gqa_eval.prompt import build_multimodal_batch_inputs

In [7]:
print(f"Evaluating split={args.split}, batch_size={gqa_loader.batch_size}, num_workers={gqa_loader.num_workers}")
dataloader = gqa_loader.as_dataloader(args.split, args.max_samples)


Evaluating split=testdev, batch_size=2, num_workers=8
加载本地 GQA split: testdev
cache_path: /home/Dataset/Dataset/GQA/.gqa_cache/testdev_arrow
use_cache: True
os.path.exists(cache_path): True
从 Arrow 缓存加载: /home/Dataset/Dataset/GQA/.gqa_cache/testdev_arrow
已加载 100 条样本（来自缓存）
DataLoader 就绪: batch_size=2, num_workers=8


In [8]:
model_to_use = model.module if hasattr(model, "module") else model

In [9]:
tokenizer.padding_side = "left"

In [10]:
# tokenizer.pad_token = tokenizer.eos_token
# model_to_use.config.pad_token_id = tokenizer.pad_token_id

In [11]:
tokenizer.padding_side

'left'

In [10]:
tokenizer.pad_token, tokenizer.eos_token, model_to_use.config.pad_token_id, tokenizer.pad_token_id

('<unk>', '</s>', 0, 0)

In [9]:
from copy import deepcopy
tmp_tokenizer_pad_token = deepcopy(tokenizer.pad_token)
tmp_tokenizer_eos_token = deepcopy(tokenizer.eos_token)
tmp_model_pad_token_id = deepcopy(model_to_use.config.pad_token_id)
tmp_tokenizer_pad_token_id = deepcopy(tokenizer.pad_token_id)


In [148]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token, tokenizer.eos_token, model_to_use.config.pad_token_id, tokenizer.pad_token_id


('</s>', '</s>', 2, 2)

In [166]:
tokenizer.pad_token = tmp_tokenizer_pad_token
tokenizer.eos_token = tmp_tokenizer_eos_token
model_to_use.config.pad_token_id = tmp_model_pad_token_id
tokenizer.pad_token_id = tmp_tokenizer_pad_token_id
tokenizer.padding_side = "right"


In [165]:
model_to_use.config.pad_token_id, tokenizer.pad_token_id, tokenizer.padding_side

(2, 0, 'right')

In [138]:
encoded = tokenizer(["hi", "hello world"], padding=True, return_tensors="pt")
print(encoded["input_ids"])
print(encoded["attention_mask"])


tensor([[    1,  7251,     0],
        [    1, 22172,  3186]])
tensor([[1, 1, 0],
        [1, 1, 1]])


In [147]:
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
model_to_use.config.pad_token_id = tokenizer.pad_token_id

In [21]:
tokenizer.padding_side = "left"

In [16]:
tokenizer.padding_side = "left"
tokenizer.padding_side

'left'

In [21]:
tokenizer.padding_side = "left"
p = ["fuck you", "rubbish shit your mom"]
tokenizer(p, padding=True)


{'input_ids': [[0, 0, 0, 0, 1, 285, 2707, 366], [1, 14051, 29890, 728, 528, 277, 596, 16823]], 'attention_mask': [[0, 0, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [42]:
tokenizer.padding_side = "left"

In [43]:
# gqa_eval/prompt.py
import torch
from concurrent.futures import ThreadPoolExecutor
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX

from llava.conversation import conv_templates
# from llava.mm_utils import tokenizer_image_token

def tokenizer_image_token(
    prompts,
    tokenizer,
    image_token_index=IMAGE_TOKEN_INDEX,
    return_tensors=None,
    padding_side="right",  # "left" or "right"
):
    """
    将文本中 <image> 替换为 image_token_index，并进行可选的 batch padding。
    - 支持单个字符串或字符串列表。
    - 支持左/右 padding。
    - 保证返回格式统一。
    """
    if isinstance(prompts, str):
        prompts = [prompts]

    tokenizer.padding_side = padding_side

    all_input_ids = []
    for prompt in prompts:
        # 按 <image> 拆分
        prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split("<image>")]
        
        # 插入 <image> 占位符
        def insert_separator(X, sep):
            return [ele for sublist in zip(X, [sep] * len(X)) for ele in sublist][:-1]

        input_ids = []
        offset = 0
        if (
            len(prompt_chunks) > 0
            and len(prompt_chunks[0]) > 0
            and prompt_chunks[0][0] == tokenizer.bos_token_id
        ):
            offset = 1
            input_ids.append(prompt_chunks[0][0])

        for x in insert_separator(prompt_chunks, [image_token_index] * (offset + 1)):
            input_ids.extend(x[offset:])

        all_input_ids.append(input_ids)
    
    # 不需要 tensor，直接返回 list[list[int]]
    if return_tensors is None:
        return all_input_ids

    # 否则返回 tensor 并自动 padding
    if return_tensors == "pt":
        padded = tokenizer.pad(
            {"input_ids": all_input_ids},
            padding=True,
            return_tensors="pt"
        )
        return padded

    raise ValueError(f"Unsupported tensor type: {return_tensors}")



def build_prompt(question: str, conv_mode: str = "llava_v1") -> str:
    """构造单样本 prompt"""
    conv = conv_templates[conv_mode].copy()
    conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
    conv.append_message(conv.roles[1], None)
    return conv.get_prompt()


def build_multimodal_batch_inputs_new(
    batch, tokenizer, image_processor, device, dtype=torch.float16, conv_mode="llava_v1", max_workers=8
):
    """
    批量构建多模态输入 (文本 + 图像)
    - 文本部分批量 tokenizer padding
    - 图像部分多线程 image_processor 预处理
    """

    # 1 文本批量生成 prompt
    prompts = []
    for s in batch:
        q = s["question"]
        conv = conv_templates[conv_mode].copy()
        conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + q)
        conv.append_message(conv.roles[1], None)
        prompts.append(conv.get_prompt())

    # 2 Tokenizer 批量编码（自动 padding）


    tok = tokenizer_image_token(
        prompts,
        tokenizer,
        IMAGE_TOKEN_INDEX,
        return_tensors="pt",
        padding_side=tokenizer.padding_side # 或 "left"
    )
    # print(tok["input_ids"])
    input_ids = tok["input_ids"]
    attention_mask = tok["attention_mask"]

    # image = batch[0]["image"]
    # image_tensor = image_processor(image, return_tensors="pt")["pixel_values"].to(device, dtype)
    # print(image_tensor)

    # 3 图像并行预处理
    def _process_img(s):
        image = s.get("image", None)
        if image is None:
            return None
        if image.mode != "RGB":
            image = image.convert("RGB")
        try:
            return image_processor(image, return_tensors="pt")["pixel_values"].to(device, dtype)
        except Exception as e:
            print(f"[warn] 图像处理失败: {e}")
            return None

    with ThreadPoolExecutor(max_workers=min(max_workers, len(batch))) as ex:
        image_tensors = list(ex.map(_process_img, batch))   

    image_sizes = [s["image"].size for s in batch]
    # print(input_ids, image_tensors)
    return input_ids, attention_mask, torch.cat(image_tensors, dim=0), image_sizes



counters = {k: Counter() for k in ["llava_match", "lmms_match", "loose_match"]}
correct_loose, total = 0, 0
results = []

for batch in tqdm(dataloader, desc="Evaluating GQA"):
    # ============================================================
    # 构建批量输入（文本+图像）
    # ============================================================


    input_ids_batch, attention_mask_batch, image_tensors, image_sizes = build_multimodal_batch_inputs_new(
        batch, tokenizer, image_processor, args.device, torch.float16, args.conv_mode
    )

    print(f"input_ids_batch: {input_ids_batch}")
    # print(f"attention_mask_batch: {attention_mask_batch}")

    # Ground truth 与问题
    gts = [s["answer"].strip().lower() for s in batch if s.get("answer")]
    questions = [s["question"] for s in batch]

    # ============================================================
    # 模型生成
    # ============================================================

    outputs = model_to_use.generate(
        inputs=input_ids_batch.to(args.device),
        images=image_tensors.to(args.device),
        max_new_tokens=args.max_new_tokens,
        attention_mask=attention_mask_batch.to(args.device),
        image_sizes=image_sizes,
        temperature=0,
        do_sample=False,
    )

    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    preds = [p.strip().lower() for p in preds]

    # ============================================================
    # 批量匹配 + 向量化统计
    # ============================================================
    batch_matches = compute_match_batch(preds, gts)

    llava_match = np.fromiter((m["llava_match"] for m in batch_matches), dtype=bool)
    lmms_match = np.fromiter((m["lmms_match"] for m in batch_matches), dtype=bool)
    loose_match = np.fromiter((m["loose_match"] for m in batch_matches), dtype=bool)

    is_correct = lmms_match | loose_match

    # 向量化统计更新
    counters["llava_match"].update({True: llava_match.sum(), False: len(llava_match) - llava_match.sum()})
    counters["lmms_match"].update({True: lmms_match.sum(), False: len(lmms_match) - lmms_match.sum()})
    counters["loose_match"].update({True: loose_match.sum(), False: len(loose_match) - loose_match.sum()})

    correct_loose += int(is_correct.sum())
    total += len(is_correct)
    # print(questions, gts)

    def single_inference(image, idx):

        single_prompt = build_prompt(questions[idx], args.conv_mode)
        tok = tokenizer_image_token([single_prompt], tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')
        single_input_ids = tok["input_ids"].to(args.device)          # shape (1, L)
        single_attention_mask = tok["attention_mask"].to(args.device)

        batch_pred = preds[idx]
        single_output = model_to_use.generate(
            inputs=single_input_ids,
            images=image.unsqueeze(0),
            image_sizes=[image.size],
            attention_mask=single_attention_mask,
            max_new_tokens=args.max_new_tokens,
            temperature=0,
            do_sample=False,
        )
        # print(f"single_prompt: {single_prompt}")
        # # print(f"batch_prompt: {tokenizer.decode(input_ids_batch[idx], skip_special_tokens=True)}")

        # print(f"batch_input_ids: {input_ids_batch[idx]}")
        # print(f"single_input_ids: {single_input_ids}")

        # print(f"batch_output: {outputs[idx]}")
        # print(f"single_output: {single_output}")

        # print(f"batch_decode: {tokenizer.batch_decode(outputs, skip_special_tokens=True)[idx]}")
        # print(f"single_decode: {tokenizer.decode(single_output[0], skip_special_tokens=True)}")

        single_pred = tokenizer.decode(single_output[0], skip_special_tokens=True).strip().lower()
        if batch_pred != single_pred:

            print("\n🔍 Debug: Batch vs Single")
            print(f"Question: {questions[idx]}")
            print(f"GT:       {gts[idx]}")
            print(f"Batch:    '{batch_pred}'")
            print(f"Single:   '{single_pred}'")
            print(f"Match?    {batch_pred == single_pred}")
    
    for i in range(len(batch)):
        single_inference(image_tensors[i], i)
    break
    


Evaluating GQA:   0%|          | 0/50 [00:00<?, ?it/s]

/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


input_ids_batch: tensor([[    0,     0,     1,   319, 13563,  1546,   263, 12758,  5199,   322,
           385, 23116, 21082, 20255, 29889,   450, 20255,  4076,  8444, 29892,
         13173, 29892,   322,  1248,   568,  6089,   304,   278,  5199, 29915,
         29879,  5155, 29889,  3148,  1001, 29901, 29871,  -200, 29871,    13,
          3624,   372,   975,  4384, 29973,   319,  1799,  9047, 13566, 29901],
        [    1,   319, 13563,  1546,   263, 12758,  5199,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  5199, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -200, 29871,    13, 22110,   338,
           591,  4362,   278, 10714, 29973,   319,  1799,  9047, 13566, 29901]])

🔍 Debug: Batch vs Single
Question: Is it overcast?
GT:       no
Batch:    '. no, it is not overcast. the sky is blue, indicating a clear and sunny day.'
Single:   'no, it is not overcast. t

Evaluating GQA:   0%|          | 0/50 [00:01<?, ?it/s]


transformers.modeling_outputs.CausalLMOutputWithPast

In [29]:
import json

def read_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)


match1_testdev_GQA_100samples_bs2_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/match1_testdev_GQA_100samples_bs2/pred_gt.json"
match1_testdev_GQA_100samples_bs2_data = read_json(match1_testdev_GQA_100samples_bs2_path)

testdev_GQA_100samples_bs2_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/testdev_GQA_100samples_bs2/pred_gt.json"
testdev_GQA_100samples_bs2_data = read_json(testdev_GQA_100samples_bs2_path)

testdev_GQA_100samples_bs2_list_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/testdev_GQA_100samples_bs2_list/pred_gt.json"
testdev_GQA_100samples_bs2_list_data = read_json(testdev_GQA_100samples_bs2_list_path)

def get_data(data):
    result = [  (d["question"], d["pred"], d["gt"]) for d in data]
    return result

match1, b2, b2l = get_data(match1_testdev_GQA_100samples_bs2_data), get_data(testdev_GQA_100samples_bs2_data), get_data(testdev_GQA_100samples_bs2_list_data)

In [55]:
match1_bs2_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/match1_testdev_GQA_100samples_bs2/pred_gt.json"
match1_bs8_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/match1_testdev_GQA_100samples_bs8/pred_gt.json"

def get_data_with_correct(data):
    result = [  (d["question"], d["pred"], d["gt"], d["is_correct"]) for d in data]
    return result

mbs2, mbs8 = get_data_with_correct(read_json(match1_bs2_path)), get_data_with_correct(read_json(match1_bs8_path))
mbs2[:3]



[('Is it overcast?', '', 'no', False),
 ('Who is wearing the dress?',
  'the image shows a woman wearing a dress.',
  'women',
  False),
 ('Does the utensil on top of the table look clean and black?',
  'yes, the utensil on top of the table looks clean and black.',
  'no',
  False)]

In [56]:
for i in range(len(mbs2)):
    a, b = mbs2[i], mbs8[i]
    assert(a[0] == b[0] and a[2] == b[2])
    if a[3] != b[3]:
        print(f' question: {a[0]}, gt: {a[2]}, match1: {a[1], }, match2: {b[1]}')



 question: Is the ground blue or brown?, gt: brown, match1: ('the ground in the image is brown.',), match2: 
 question: Is the jacket long sleeved and black?, gt: yes, match1: ('yes, the jacket is long sleeved and black.',), match2: 
 question: Are there beds next to the small outlet?, gt: yes, match1: ('yes, there are beds next to the small outlet.',), match2: 
 question: Are the plates on top of an ottoman?, gt: no, match1: ('no, the plates are not on top of an ottoman. they are on top of a table.',), match2: 
 question: Which side are the white houses on?, gt: left, match1: ('left',), match2: 
 question: Are there either any small refrigerators or microwaves in the picture?, gt: no, match1: ('no, there are no small refrigerators or microwaves in the picture. the image shows a large refrigerator and a microwave.',), match2: yes, there is a small refrigerator and a small microwave in the picture.
 question: How does that car look like, orange or maybe white?, gt: white, match1: ('the 

In [34]:
match1[:5]

[('Is it overcast?', '', 'no'),
 ('Who is wearing the dress?',
  'the image shows a woman wearing a dress.',
  'women'),
 ('Does the utensil on top of the table look clean and black?',
  'yes, the utensil on top of the table looks clean and black.',
  'no'),
 ('Is the surfer that looks wet wearing a wetsuit?', '(no)', 'yes'),
 ('How tall is the chair in the bottom of the photo?',
  'the chair in the bottom of the photo is very tall.',
  'short')]

In [35]:
match1_testdev_GQA_100samples_bs8_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/match1_testdev_GQA_100samples_bs8/pred_gt.json"
match1_testdev_GQA_100samples_bs8_data = read_json(match1_testdev_GQA_100samples_bs8_path)

testdev_GQA_100samples_bs8_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/testdev_GQA_100samples_bs8/pred_gt.json"
testdev_GQA_100samples_bs8_data = read_json(testdev_GQA_100samples_bs8_path)

testdev_GQA_100samples_bs8_list_path = r"/home/czj/llava15_test/LLaVA/eval_GQA_results/testdev_GQA_100samples_bs8_list/pred_gt.json"
testdev_GQA_100samples_bs8_list_data = read_json(testdev_GQA_100samples_bs8_list_path)

match2, b8, b8l = get_data(match1_testdev_GQA_100samples_bs8_data), get_data(testdev_GQA_100samples_bs8_data), get_data(testdev_GQA_100samples_bs8_list_data)



In [36]:
match2 == b8, match2 == b8l

(True, True)

In [37]:
match2[:5]

[('Is it overcast?', '', 'no'),
 ('Who is wearing the dress?', '', 'women'),
 ('Does the utensil on top of the table look clean and black?',
  'yes, the utensil on top of the table looks clean and black.',
  'no'),
 ('Is the surfer that looks wet wearing a wetsuit?', '(no)', 'yes'),
 ('How tall is the chair in the bottom of the photo?', '', 'short')]

In [38]:
for i in range(len(match2)):
    a, b = match1[i], match2[i]
    assert(a[0] == b[0] and a[2] == b[2])
    if a != b:
        print(f' question: {a[0]}, gt: {a[2]}, match1: {a[1], }, match2: {b[1]}')




 question: Who is wearing the dress?, gt: women, match1: ('the image shows a woman wearing a dress.',), match2: 
 question: How tall is the chair in the bottom of the photo?, gt: short, match1: ('the chair in the bottom of the photo is very tall.',), match2: 
 question: What kind of device is on top of the desk?, gt: keyboard, match1: ('there is a laptop computer on top of the desk.',), match2: 
 question: What is the airplane flying above?, gt: ocean, match1: ('the airplane is flying above a city.',), match2: 
 question: What color are the pants?, gt: red, match1: ('<image>',), match2: 
 question: Is the ground blue or brown?, gt: brown, match1: ('the ground in the image is brown.',), match2: 
 question: What is around the open window?, gt: drapes, match1: ('there is a chair placed next to the open window.',), match2: 
 question: What's around the window?, gt: drapes, match1: ('there is a white frame around the window.',), match2: 
 question: Who is standing at the table?, gt: woman, 

In [18]:
import pickle

CACHE_FILE = "gqa_cache.pkl"
if Path(CACHE_FILE).exists():
    print("✅ 从缓存加载数据样本...")
    with open(CACHE_FILE, "rb") as f:
        dataset_cache = pickle.load(f)
else:
    print("🚀 第一次加载数据集...")
    gqa_loader = GQALoader(gqa_root="/home/Dataset/Dataset/GQA")
    dataset = gqa_loader.load_dataset(split="train_balanced", num_samples=10)
    dataset_cache = [gqa_loader.process_sample(dataset[i]) for i in range(min(10, len(dataset)))]

    
    # 存缓存
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(dataset_cache, f)
    print("✅ 已缓存前10个样本到 gqa_cache.pkl")

print(f"缓存样本数: {len(dataset_cache)}")
sample = dataset_cache[0]
sample.keys()


✅ 从缓存加载数据样本...
缓存样本数: 10


dict_keys(['id', 'imageId', 'question', 'answer', 'fullAnswer', 'types', 'groups', 'semantic', 'structural', 'image_data', 'image'])

In [57]:
model_path = "/home/czj/llava15_test/llava-v1.5-7b"
clip_path = "/home/czj/llava15_test/clip-vit-large-patch14-336"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    device_map="cuda:1",
    torch_dtype=torch.float16,
    use_masked_model=True
)

print("✅ 模型加载完成")


You are using a model of type llava to instantiate a model of type llava_llama_masked. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.87s/it]


✅ 模型加载完成


In [58]:
type(model)

llava.model.language_model.llava_llama_masked.LlavaLlamaForCausalLMMasked

In [21]:
sample = dataset_cache[0]
image = sample["image"].convert("RGB")

image_tensor = image_processor(image, return_tensors="pt")["pixel_values"].to(model.device, dtype=torch.float16)

print(f"[DEBUG] 原图尺寸: {image.size}, tensor形状: {image_tensor.shape}")


[DEBUG] 原图尺寸: (500, 270), tensor形状: torch.Size([1, 3, 336, 336])


In [59]:
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import tokenizer_image_token

for sample in dataset_cache:
    image = sample["image"].convert("RGB")

    image_tensor = image_processor(image, return_tensors="pt")["pixel_values"].to(model.device, dtype=torch.float16)
    question = sample["question"]

    conv = conv_templates["llava_v1"].copy()
    conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            inputs=input_ids,
            images=[image_tensor],
            max_new_tokens=64,
            do_sample=False,
            temperature=0,
            top_p=None,
            num_beams=1
        )


    # answer = tokenizer.batch_decode(output_tokens[0], skip_special_tokens=True)
    answer = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print("问题：", question, " 模型输出:", answer)



/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


问题： Is the sky dark?  模型输出: Yes, the sky is dark in the image.
问题： What is on the white wall?  模型输出: There is a painting on the white wall.
问题： Is that pipe red?  模型输出: No, the pipe is not red. It is a black pipe.
问题： Is the tall clock small or large?  模型输出: The tall clock is large, as it is described as a large clock tower.
问题： Who is wearing a shirt?  模型输出: A boy is wearing a shirt.
问题： What do you think is he sleeping in?  模型输出: The man is sleeping in a bed, which is covered with a white comforter.
问题： Is the cheese to the left of the food on the plate?  模型输出: Yes, the cheese is to the left of the food on the plate.
问题： What is the piece of furniture that he is sleeping in?  模型输出: The man is sleeping in a bed.
问题： What kind of furniture is to the right of the chair?  模型输出: There is a couch to the right of the chair.
问题： Is the steel spatula in the top of the image?  模型输出: Yes, the steel spatula is in the top of the image.


In [65]:
model.model.mask_visual_token

False

In [44]:
from llava.model.builder import load_pretrained_model
from llava.conversation import conv_templates
from llava.mm_utils import tokenizer_image_token, process_images
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
import torch
from PIL import Image


# # 载入模型
# tokenizer, model, image_processor, context_len = load_pretrained_model(
#     model_path=model_path,
#     model_base=None,
#     model_name=get_model_name_from_path(model_path),
#     device_map="cuda:1",
#     torch_dtype=torch.float16,
#     # use_masked_model=True
# )
# 图像
image = Image.open("/home/czj/llava15_test/LLaVA/images/llava_v1_5_radar.jpg").convert("RGB")
image_tensor = process_images([image], image_processor, model.config).to(model.device, dtype=torch.float16)

# prompt
question = "What is the color of the object in the image?"
conv = conv_templates["llava_v1"].copy()
conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)

# 生成
output = model.generate(
    input_ids,
    images=image_tensor,
    max_new_tokens=64,
    do_sample=False,
    temperature=0,
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


/home/czj/anaconda3/envs/llava15/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The object in the image is red.


In [45]:
output

tensor([[    1,   450,  1203,   297,   278,  1967,   338,  2654, 29889,     2]],
       device='cuda:1')

In [28]:
model.eval()
with torch.no_grad():
    outputs = model(
        input_ids=input_ids,
        images=image_tensor,
        output_hidden_states=True,
        return_dict=True,
    )

print(f"[DEBUG] Last hidden shape: {outputs.hidden_states[-1].shape}")


[DEBUG] Last hidden shape: torch.Size([1, 623, 4096])


In [29]:
with torch.no_grad():
    output_tokens = model.generate(
        inputs=input_ids,
        images=image_tensor,
        max_new_tokens=64,
        do_sample=False,
        temperature=0,
        top_p=None,
        num_beams=1
    )

answer = tokenizer.batch_decode(output_tokens[0], skip_special_tokens=True)[0]
print("🧩 模型输出:", answer)


🧩 模型输出: <s>
